In [3]:
#!/usr/bin/env python
# Jupyter Notebook 2: Retrieve & Insert Updated Clash Royale Data

import datetime
import json
import os
import pandas as pd
import requests
import urllib.parse
import hopsworks
import warnings
warnings.filterwarnings("ignore")

# ------------------------------------------------------------------------------
# 1. SET UP ENVIRONMENT AND HOPSWORKS LOGIN
# ------------------------------------------------------------------------------
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment below and read your key from a file or set it manually.
# os.environ["HOPSWORKS_API_KEY"] = 'YOUR-HOPSWORKS-API-KEY'

#Read your Hopsworks API Key
with open('../data/hopsworks-api-key.txt', 'r') as f:
    api_key = f.read().strip()

# 2. Set the environment variable for Hopsworks
os.environ["HOPSWORKS_API_KEY"] = api_key
project = hopsworks.login()
fs = project.get_feature_store()

2025-01-05 19:36:08,714 INFO: Initializing external client
2025-01-05 19:36:08,714 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-05 19:36:10,368 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1175700


In [4]:

# ------------------------------------------------------------------------------
# 2. RETRIEVE SECRETS (IF USING HOPSWORKS SECRETS)
# ------------------------------------------------------------------------------
# If you have stored your Clash Royale API key as a secret in Hopsworks, do the following:
# secrets = util.secrets_api(project.name)
# CR_API_KEY = secrets.get_secret("CLASH_ROYALE_API_KEY").value

# OR read your API key locally (like in Notebook 1)
local_key_file = '../data/clash-royale-api-key.txt'
with open(local_key_file, 'r') as file:
    CR_API_KEY = file.read().strip()

In [5]:

# ------------------------------------------------------------------------------
# 3. DEFINE ANY HELPER FUNCTIONS (IF NOT IN 'util.py')
# ------------------------------------------------------------------------------
def get_player_data(player_tag, api_key):
    """
    Fetch the latest battle log for a given player from the Clash Royale API.
    Return the JSON data if successful, otherwise None.
    """
    # URL-encode the player tag (e.g., #2LGY9G -> %232LGY9G)
    url = f'https://api.clashroyale.com/v1/players/{player_tag}/battlelog'
    headers = {
        'Accept': 'application/json',
        'Authorization': f'Bearer {api_key}'
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f'Error: {response.status_code}, check your player tag or API key.')
        return None

def clean_battle_data(battle_data):
    """
    Clean and filter the raw JSON battle data.
    Return a list (or any structure) suitable for DataFrame creation.
    """
    # (Same logic from Notebook 1 or adapted as needed)
    # ...
    # For brevity, we’ll assume you copy the same code from Notebook 1:
    desired_keys = {'name', 'level', 'elixirCost', 'rarity'}
    def filter_card_data(card):
        return {key: card[key] for key in card if key in desired_keys}
    def process_team_or_opponent(team_or_opponent):
        team_or_opponent['cards'] = [filter_card_data(card) for card in team_or_opponent['cards']]
        return team_or_opponent
    for battle in battle_data:
        battle['team'] = [process_team_or_opponent(member) for member in battle['team']]
        battle['opponent'] = [process_team_or_opponent(member) for member in battle['opponent']]
    return battle_data

def create_dataframe_from_battle_data(battle_data):
    """
    Transform the cleaned battle data into a DataFrame with desired columns.
    """
    # Example from Notebook 1
    import pandas as pd

    # Dictionary mapping card names to IDs
    cards_dict = {
        "Archers": 1, "Archer Queen": 2, "Baby Dragon": 3, "Balloon": 4, "Bandit": 5, "Barbarians": 6,
        "Bats": 7, "Battle Healer": 8, "Battle Ram": 9, "Bomber": 10, "Bowler": 11, "Bush Goblins": 12,
        "Cannon Cart": 13, "Cursed Hog": 14, "Dark Prince": 15, "Dart Goblin": 16, "Electro Dragon": 17,
        "Electro Giant": 18, "Electro Spirit": 19, "Electro Wizard": 20, "Elite Barbarians": 21,
        "Elixir Blob": 22, "Elixir Golem": 23, "Elixir Golemite": 24, "Executioner": 25, "Firecracker": 26,
        "Fire Spirit": 27, "Fisherman": 28, "Flying Machine": 29, "Giant": 30, "Giant Skeleton": 31,
        "Goblin Brawler": 32, "Goblin Gang": 33, "Goblin Demolisher": 34, "Goblin Giant": 35,
        "Goblin Machine": 36, "Goblins": 37, "Goblinstein": 38, "Golden Knight": 39, "Golem": 40,
        "Golemite": 41, "Guardienne": 42, "Guards": 43, "Hog Rider": 44, "Hunter": 45, "Heal Spirit": 46,
        "Ice Golem": 47, "Ice Spirit": 48, "Ice Wizard": 49, "Inferno Dragon": 50, "Knight": 51,
        "Lava Hound": 52, "Lava Pup": 53, "Little Prince": 54, "Lumberjack": 55, "Magic Archer": 56,
        "Mega Knight": 57, "Mega Minion": 58, "Mighty Miner": 59, "Miner": 60, "Mini P.E.K.K.A.": 61,
        "Minion Horde": 62, "Minions": 63, "Monk": 64, "Mother Witch": 65, "Monster": 66, "Musketeer": 67,
        "Night Witch": 68, "P.E.K.K.A.": 69, "Phoenix": 70, "Reborn Phoenix": 71, "Prince": 72,
        "Princess": 73, "Ram Rider": 74, "Rascal Boy": 75, "Rascal Girl": 76, "Royal Ghost": 77,
        "Royal Giant": 78, "Royal Hogs": 79, "Royal Recruits": 80, "Skeleton Army": 81,
        "Skeleton Barrel": 82, "Skeleton Dragons": 83, "Skeleton King": 84, "Skeletons": 85, "Sparky": 86,
        "Spear Goblins": 87, "Suspicious Bush": 88, "Three Musketeers": 89, "Valkyrie": 90,
        "Wall Breakers": 91, "Witch": 92, "Wizard": 93, "Zappies": 94,"Bomb Tower": 95, "Cannon": 96, "Cannon Cart (broken)": 97, "Inferno Tower": 98, "Mortar": 99,
        "Tesla": 100, "X-Bow": 101,"Barbarian Hut": 102, "Elixir Collector": 103, "Furnace": 104, "Goblin Cage": 105,
        "Goblin Drill": 106, "Goblin Hut": 107, "Phoenix Egg": 108, "Tombstone": 109, "Arrows": 110, "Barbarian Barrel": 111, "Earthquake": 112, "Fireball": 113, "Freeze": 114,
        "Giant Snowball": 115, "Goblin Curse": 116, "Lightning": 117, "Poison": 118, "Rage": 119, "Rocket": 120, "Royal Delivery": 121, "The Log": 122, "Tornado": 123, "Void": 124, "Zap": 125,
        "Barbarian Barrel": 126, "Barbarian Hut": 127, "Battle Ram": 128, "Elixir Golem": 129,
        "Elixir Golemite": 130, "Furnace": 131, "Goblin Barrel": 132, "Goblin Cage": 133, "Goblin Curse": 134,
        "Goblin Drill": 135, "Goblin Giant": 136, "Goblin Hut": 137, "Golem": 138, "Graveyard": 139,
        "Lava Hound": 140, "Little Prince": 141, "Mother Witch": 142, "Night Witch": 143,
        "Phoenix Egg": 144, "Royal Delivery": 145, "Skeleton Barrel": 146, "Skeleton King": 147,
        "Suspicious Bush": 148, "Tombstone": 149, "Witch": 150,
        "Archers/Evolution": 155, "Barbarians/Evolution": 156, "Battle Ram/Evolution": 157,
        "Bats/Evolution": 158, "Bomber/Evolution": 159, "Cannon/Evolution": 160,
        "Electro Dragon/Evolution": 161, "Firecracker/Evolution": 162, "Giant Snowball/Evolution": 163,
        "Goblin Barrel/Evolution": 164, "Goblin Cage/Evolution": 165, "Goblin Drill/Evolution": 166,
        "Goblin Giant/Evolution": 167, "Ice Spirit/Evolution": 168, "Knight/Evolution": 169,
        "Mega Knight/Evolution": 170, "Mortar/Evolution": 171, "Musketeer/Evolution": 172,
        "P.E.K.K.A/Evolution": 173, "Royal Giant/Evolution": 174, "Royal Recruits/Evolution": 175,
        "Skeletons/Evolution": 176, "Tesla/Evolution": 177, "Valkyrie/Evolution": 178,
        "Wall Breakers/Evolution": 179, "Wizard/Evolution": 180, "Zap/Evolution": 181
    }

    def process_player_data(player, is_team=True):
        deck = [cards_dict.get(card['name'], -1) for card in player['cards']]
        # Example: store each card in a separate column
        return {
            'player_name': player.get('name', 'Unknown'),
            **{f'Card_{i+1}': c_id for i, c_id in enumerate(deck)},
            # Example: store result or other info here as well
        }

    all_matches = []
    for match in battle_data:
        team = match['team'][0]
        opponent = match['opponent'][0]

        team_data = process_player_data(team, is_team=True)
        opponent_data = process_player_data(opponent, is_team=False)

        # Determine match result
        if team['crowns'] > opponent['crowns']:
            team_data['Result'] = 'WIN'
            opponent_data['Result'] = 'LOSE'
        elif team['crowns'] < opponent['crowns']:
            team_data['Result'] = 'LOSE'
            opponent_data['Result'] = 'WIN'
        else:
            team_data['Result'] = 'DRAW'
            opponent_data['Result'] = 'DRAW'

        all_matches.append(team_data)
        all_matches.append(opponent_data)

    df = pd.DataFrame(all_matches)
    return df

In [6]:

# ------------------------------------------------------------------------------
# 4. FETCH THE UPDATED DATA FROM THE CLASH ROYALE API
# ------------------------------------------------------------------------------
# Suppose you have multiple player tags or just one
player_tags = ['%232LGY9G']  # Example. Add more as needed.

all_players_df_list = []
for p_tag in player_tags:
    raw_data = get_player_data(p_tag, CR_API_KEY)
    if raw_data:
        cleaned_data = clean_battle_data(raw_data)
        df_player = create_dataframe_from_battle_data(cleaned_data)
        all_players_df_list.append(df_player)

# Concatenate all players' data into one DataFrame
if all_players_df_list:
    final_df = pd.concat(all_players_df_list, ignore_index=True)
else:
    final_df = pd.DataFrame()
    
print("Preview of the final DataFrame to insert:")
display(final_df.head())

Preview of the final DataFrame to insert:


,Player_Name,Card_1,Card_2,Card_3,Card_4,Card_5,Card_6,Card_7,Card_8,Result
0,Diwel26,78,26,126,28,113,147,142,149,LOSE
1,حسین شهبازی,93,120,38,81,6,57,90,4,WIN
2,Diwel26,78,26,126,28,113,147,142,149,LOSE
3,killergamer,57,7,5,110,2,72,91,125,WIN
4,Diwel26,78,26,126,28,113,147,142,149,LOSE


In [ ]:
# ------------------------------------------------------------------------------
# 5. INSERT THE NEW DATA INTO THE HOPSWORKS FEATURE STORE
# ------------------------------------------------------------------------------
if not final_df.empty:
    # Get or create the feature group
    cr_feature_group = fs.get_or_create_feature_group(
        name="clash_royale_features",
        version=1,
        description="Clash Royale updated match features",
        primary_key=["player_name"]  # adapt if needed
    )

    # Insert the new data
    cr_feature_group.insert(final_df)
    print("DataFrame successfully inserted into 'clash_royale_features'!")
else:
    print("No new data to insert (DataFrame is empty).")
